#**AI voice phishing prevention system using STT API for the elderly and the weak**
######Edit data : 2021-09-16
######*2021 Probono Project*

#**0. Requirement (import)**

In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install JPype1-py3
!pip3 install konlpy
!JAVA_HOME="C:\Program Files\Java\jdk-16.0.1"

from konlpy.tag import Okt
okt = Okt()

import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from random import *
from sklearn.datasets import load_digits
import nltk
nltk.download('punkt')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad

#**1. KOR_EDA.py 모듈, yes,not 증량**

In [ ]:
import sys
sys.path.append('/content/KorEDA_yes.py')
sys.path.append('/content/KorEDA_not.py')

In [ ]:
import KorEDA_yes as KEDA_yes
import KorEDA_not as KEDA_not

In [ ]:
#yes Aug
yesvpDataSet = pd.read_csv("/content/yesvp.txt" , names=['data', 'label'], sep='|')
yesvpTrainData = yesvpDataSet['data'].to_numpy()
yesvpTargetData = yesvpDataSet['label'].to_numpy()

In [ ]:
yesDataBody=[]

print("data set count : ",end='')

for sample in yesvpTrainData:
    for n in KEDA_yes.KOR_EDA(sample):
        yesDataBody.append(n)
print(len(yesDataBody))
print(yesDataBody[0:1])
#num=62->10044

In [ ]:
#not Aug
notvpDataSet = pd.read_csv('notvp.txt', names=['text'])
notvpTrainData = notvpDataSet['text'].to_numpy()

#데이터셋개수줄이기
num = 0
notvpTrainDataRed = []
for k in range(1112):
  a = ''
  num += 50
  for i in range(num-50, num):
    sentence = notvpTrainData[i]
    a = a + sentence
  notvpTrainDataRed.append(a)

notvpTrainDataRed=np.array(notvpTrainDataRed) #list to array (now, body=numpy)
print(len(notvpTrainData),"->",len(notvpTrainDataRed))

In [ ]:
notDataBody=[]

print("data set count : ",end='')

for sample in notvpTrainDataRed:
    for n in KEDA_not.KOR_EDA(sample):
        notDataBody.append(n)
print(len(notDataBody))
print(notDataBody[0:1])
#num=9->10008

#**2. yesvp + notvp 데이터셋 만들기**

In [ ]:
yesvpTrainDataNew=yesDataBody
notvpTrainDataNew=notDataBody

In [ ]:
allTrainData=np.append(yesvpTrainDataNew,notvpTrainDataNew)
print(len(allTrainData))

In [ ]:
print(allTrainData[:])

#**3. (ALL) Target Data**

In [ ]:
print(len(yesvpTrainData))
print(len(yesvpTrainDataNew))
print(len(yesvpTargetData))

In [ ]:
#yes 데이터 라벨값 증량 개발

print(len(yesvpTargetData))
print(type(yesvpTargetData))
print(yesvpTargetData)

yesvpTargetDataNew=np.ones(10044)

print(len(yesvpTargetDataNew))
print(type(yesvpTargetDataNew))
print(yesvpTargetDataNew)


In [ ]:
print(len(notvpTrainDataRed))
print(len(notvpTrainDataNew))

In [ ]:
#라벨링 이거 이렇게 하는거 맞나? (5번에 있는거)
notvpTargetDataNew=np.zeros(10008)
print(notvpTargetDataNew)
print(len(notvpTargetDataNew))
print(type(notvpTargetDataNew))

In [ ]:
allTargetData=np.append(yesvpTargetDataNew,notvpTargetDataNew) 
print(len(allTargetData))

In [ ]:
print(len(allTrainData))
print(len(allTargetData))

#**4. 전처리(Preprocessing)**

In [ ]:
import time
start = time.time()

# allTrainData 토큰화
tokenize = []
num=0
for text in allTrainData:
  num+=1
  tokenSet = okt.morphs(text)
  tokenize.append(tokenSet)
  if num%1000==0:
    print(num,"/",len(allTrainData),"->",time.time()-start)
print(num,"/",len(allTrainData),"->",time.time()-start)

In [ ]:
stop_words = pd.read_csv('stop_words_new.txt',names = ['words'], encoding='utf-16')
stop_data = np.array(stop_words['words']).tolist()

In [ ]:
# 불용어처리
stop_result = []
stop_body = []
for tokenSet in tokenize:
  for token in tokenSet:
    if token not in stop_data:
      stop_result.append(token)
  stop_body.append(stop_result)
  stop_result=[]

#result엔 뭐가 들어있는거지 -> result는 샘플 하나의 불용어 처리한 결과가, body는 그런 result들이 append되서 만들어짐.
print("토큰화 한 샘플(tokenize) = ",tokenize[0]) #토큰화 한 샘플
print("불용어 처리 된 샘플(body) = ",stop_body[0]) #불용어 처리 된 샘플
print(len(tokenize[0]))
print(len(stop_body[0]))
#문제: 숫자랑 문자로 표현된 숫자 불용어 처리 어떻게 할 건지

In [ ]:
#어휘사전 만들기
t = Tokenizer()
t.fit_on_texts(stop_body)
print(t.word_index)
print(len(t.word_index))

In [ ]:
keys = []
for key in t.word_index.keys():
  keys.append(key)
print(keys[500])

In [ ]:
keys = keys[:500]
#print(len(keys))
dictionary = {keys[i]:i for i in range(len(keys))}
print(dictionary)
t.word_index = dictionary

In [ ]:
#transform
sequence_data = t.texts_to_sequences(stop_body) #transform = 숫자화 
print('0=',sequence_data[0])

In [ ]:
#단어사전 줄이기 
dicList = []
for sequenceSet in sequence_data:
  temp = []
  for sequence in sequenceSet:
    if sequence > 499:
      sequence = 499
    temp.append(sequence)
  dicList.append(temp)
print(dicList[0])
print(len(t.word_index))

In [ ]:
#Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(allTrainData)

In [ ]:
encoded = tokenizer.texts_to_sequences(allTrainData)
print(encoded)

In [ ]:
padded = pad_sequences(dicList, truncating = 'post', maxlen = 500)
#print(padded[0])
print(padded.shape)

In [ ]:
#이거 뭔 코드예요?
body = []
for size in range(9900):
    randomList = []
    for i in range(500):
        randomList.append(randint(1,6800))
    body.append(randomList)

body = np.array(body)

final = np.append(padded, body, axis=0)
print(final.shape)

#**6. 훈련데이터셋분할**

In [ ]:
x = allTrain
y = allTargetData

In [ ]:
#훈련데이터셋 분할
from sklearn.model_selection import train_test_split

#dataset 자체는 order가 깔끔히 정렬된 상태이기 때문에 반드시 dataset을 shuffle 해주어야 한다.
#sklearn에서 제공해주는 train_test_split을 활용하면 매우 쉽게 shuffle과 split을 할 수 있다.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

#train_test_split()은 기본적으로 훈련데이터 세트를 75:25의 비율로 나눔 
#이 비율을 조정하고 싶을 때는 test_size 매개변수에 테스트 세트의 비율을 전달하면 조절할 수 있음 
#여기서는 입력된 데이터 세트의 20%를 테스트 세트로 나누기 위해 0.2를 전달